# New New

In [ ]:
# Cell 1
!pip install geopandas matplotlib descartes


In [4]:
# Cell 2
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import warnings
import sys
from io import StringIO

# Suppress specific warnings
warnings.filterwarnings("ignore", message="Expecting property name enclosed in double quotes: line 1 column 2 (char 1)", module="fiona.ogrext")

# Create a custom stream class to filter out specific messages
class FilteredStream:
    def __init__(self, stream):
        self.stream = stream
        self.buffer = StringIO()

    def write(self, data):
        if "WARNING:fiona.ogrext:Expecting property name enclosed in double quotes" not in data:
            self.stream.write(data)
        else:
            self.buffer.write(data)  # Capture discarded data if needed

    def flush(self):
        self.stream.flush()

# Replace sys.stderr with our filtered stream
sys.stderr = FilteredStream(sys.stderr)

# List of input GeoJSON file paths
input_paths = [
    "/geoJSON/cleaned_weighted/Agricultural_Minerals_Operations_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Aquifers_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Aviation_Facilities_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Biodiesel_Plants_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Bridges_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/CBP_2021_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Cellular_Towers_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Colleges_and_Universities_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Construction_Minerals_Operations_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Courthouses_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/DOE_Petroleum_Reserves_Cleaned.geojson",
    "/geoJSON/cleaned_weighted/Dams_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Dialysis_Centers_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/EMS_Stations_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Ethanol_Plants_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/FDIC_Banks_Cleaned_weighted_cleaned.geojson",
    # "/geoJSON/cleaned/Federal_Reserve_Cleaned.geojson",
    "/geoJSON/cleaned_weighted/Ferrous_Metal_Mines_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Ferrous_Metal_Processing_Plants_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Fortune_500_HQ_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Gas_Compressor_Stations_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Gold_Repositories_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Hospitals_Weighted.geojson",
    "/geoJSON/cleaned_weighted/Hydrocarbon_Pipelines_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Industrial_Mineral_Operations_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/LNG_Storage_Facilities_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/LNG_Terminals_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Local_Law_Enforcement_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Manufacturing_Facilities_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Microwave_Service_Towers_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Military_Installations_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Mines_and_Mineral_Resources_Cleaned_weighted_cleaned.geojson",
    # "/geoJSON/cleaned/Natural_Gas_Wells_Cleaned.geojson",
    "/geoJSON/cleaned_weighted/NGL_Pipelines_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/NG_Processing_Plants_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Nonferrous_Metal_Mines_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Oil_Refineries_Cleaned_weighted_cleaned.geojson",
    # "/geoJSON/cleaned/Oil_Wells_Cleaned.geojson",
    "/geoJSON/cleaned_weighted/Peak_Shaving_Facilities_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Petroleum_Terminals_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Pharmacies_Weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Power_Transmission_Lines_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Primary_Roads_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Principal_Ports_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Private_Schools_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Public_Health_Departments_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Rail_Lines_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Refrigerated_Warehouses_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Sand_and_Gravel_Operations_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Solid_Waste_Facilities_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Spaceports_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/State_Capitol_Buildings_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/State_Government_Buildings_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/USACE_Offices_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/USACE_Reservoirs_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Uranium_and_Vanadium_Deposits_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/Urgent_Care_Facilities_Cleaned_weighted_cleaned.geojson",
    "/geoJSON/cleaned_weighted/VA_Medical_Facilities_Cleaned_weighted_cleaned.geojson"
]


In [ ]:
# Cell 3
# Dictionary to hold data and color mappings
data = {}
colors = plt.cm.tab20.colors

# Load data and assign a color to each dataset
print("Loading datasets...")
for i, path in enumerate(input_paths):
    try:
        print(f"\nLoading {os.path.basename(path)} from {path}...")
        gdf = gpd.read_file(path)
        # Ensure the data is in the correct CRS
        if gdf.crs != "EPSG:4326":
            gdf = gdf.to_crs("EPSG:4326")
        data[path] = {'gdf': gdf, 'color': colors[i % len(colors)]}
        print(f"  - {os.path.basename(path)} loaded successfully.")
        print(f"  - GeoDataFrame head for {os.path.basename(path)}:\n{gdf.head()}")
    except Exception as e:
        print(f"  - Error loading {os.path.basename(path)}: {e}")


print("All datasets loaded.")

# Load the US political boundaries
print("Loading US political boundaries...")
try:
    us_boundaries = gpd.read_file("/datasets/raw/geographic/gadm41_USA_2.json")
    if us_boundaries.crs != "EPSG:4326":
        us_boundaries = us_boundaries.to_crs("EPSG:4326")
    print("US political boundaries loaded successfully.")
except Exception as e:
    print(f"Error loading US political boundaries: {e}")

In [ ]:
# Cell 4
# Plotting the full map without the legend
print("Creating the full map without the legend...")

# Check if us_boundaries is empty or has invalid geometries
if us_boundaries.empty:
    print("US political boundaries GeoDataFrame is empty. Aborting plot.")
else:
    try:
        fig, ax = plt.subplots(1, 1, figsize=(15, 15))

        # Plot US boundaries
        print("Plotting US political boundaries...")
        us_boundaries.plot(ax=ax, color='lightgray')

        # Plot each dataset with assigned color
        for path, info in data.items():
            print(f"Plotting {os.path.basename(path)}...")
            info['gdf'].plot(ax=ax, color=info['color'], label=os.path.basename(path))
            print(f"  - {os.path.basename(path)} plotted.")

        # Title and labels
        print("Adding title and labels...")
        plt.title('Critical Infrastructure in the United States', fontsize=20)
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')

        # Save the full map without the legend
        print("Saving the full map without the legend...")
        plt.savefig("/content/drive/MyDrive/CI_Sentinal/geoJSON/maps/full_map_without_legend.png")
        plt.close(fig)
        print("Full map saved.")
    except Exception as e:
        print(f"Error while plotting: {e}")

In [ ]:
# Cell 5
# Creating a separate image for the legend
print("Creating a separate image for the legend...")

# Extract handles and labels from the last plot
fig_legend, ax_legend = plt.subplots(figsize=(15, 10))
ax_legend.axis('off')

try:
    # Attempt to extract legend information
    handles, labels = ax.get_legend_handles_labels()
    if handles and labels:
        fig_legend.legend(handles, labels, loc='center', fontsize='small', title='Datasets', ncol=3)
        fig_legend.savefig("/content/drive/MyDrive/CI_Sentinal/geoJSON/maps/legend.png")
        plt.close(fig_legend)
        print("Legend image saved.")
    else:
        print("No handles or labels found for the legend.")
except Exception as e:
    print(f"Error while creating the legend image: {e}")

#### States and Counties List

In [8]:
# List of states and counties, alphabetically
states_counties = {
    "Alabama": [
        "Autauga",
        "Baldwin",
        "Barbour",
        "Bibb",
        "Blount",
        "Bullock",
        "Butler",
        "Calhoun",
        "Chambers",
        "Cherokee",
        "Chilton",
        "Choctaw",
        "Clarke",
        "Clay",
        "Cleburne",
        "Coffee",
        "Colbert",
        "Conecuh",
        "Coosa",
        "Covington",
        "Crenshaw",
        "Cullman",
        "Dale",
        "Dallas",
        "DeKalb",
        "Elmore",
        "Escambia",
        "Etowah",
        "Fayette",
        "Franklin",
        "Geneva",
        "Greene",
        "Hale",
        "Henry",
        "Houston",
        "Jackson",
        "Jefferson",
        "Lamar",
        "Lauderdale",
        "Lawrence",
        "Lee",
        "Limestone",
        "Lowndes",
        "Macon",
        "Madison",
        "Marengo",
        "Marion",
        "Marshall",
        "Mobile",
        "Monroe",
        "Montgomery",
        "Morgan",
        "Perry",
        "Pickens",
        "Pike",
        "Randolph",
        "Russell",
        "SaintClair",
        "Shelby",
        "Sumter",
        "Talladega",
        "Tallapoosa",
        "Tuscaloosa",
        "Walker",
        "Washington",
        "Wilcox",
        "Winston"
    ],
    "Alaska": [
        "AleutiansEast",
        "AleutiansWest",
        "Anchorage",
        "Bethel",
        "BristolBay",
        "Denali",
        "Dillingham",
        "FairbanksNorthStar",
        "Haines",
        "Juneau",
        "KenaiPeninsula",
        "KetchikanGateway",
        "KodiakIsland",
        "LakeandPeninsula",
        "Matanuska-Susitna",
        "Nome",
        "NorthSlope",
        "NorthwestArctic",
        "PrinceofWales-OuterKetchi",
        "Sitka",
        "Skagway-Yakutat-Angoon",
        "SoutheastFairbanks",
        "Valdez-Cordova",
        "WadeHampton",
        "Wrangell-Petersburg",
        "Yukon-Koyukuk"
    ],
    "Arizona": [
        "Apache",
        "Cochise",
        "Coconino",
        "Gila",
        "Graham",
        "Greenlee",
        "LaPaz",
        "Maricopa",
        "Mohave",
        "Navajo",
        "Pima",
        "Pinal",
        "SantaCruz",
        "Yavapai",
        "Yuma"
    ],
    "Arkansas": [
        "Arkansas",
        "Ashley",
        "Baxter",
        "Benton",
        "Boone",
        "Bradley",
        "Calhoun",
        "Carroll",
        "Chicot",
        "Clark",
        "Clay",
        "Cleburne",
        "Cleveland",
        "Columbia",
        "Conway",
        "Craighead",
        "Crawford",
        "Crittenden",
        "Cross",
        "Dallas",
        "Desha",
        "Drew",
        "Faulkner",
        "Franklin",
        "Fulton",
        "Garland",
        "Grant",
        "Greene",
        "Hempstead",
        "HotSpring",
        "Howard",
        "Independence",
        "Izard",
        "Jackson",
        "Jefferson",
        "Johnson",
        "Lafayette",
        "Lawrence",
        "Lee",
        "Lincoln",
        "LittleRiver",
        "Logan",
        "Lonoke",
        "Madison",
        "Marion",
        "Miller",
        "Mississippi",
        "Monroe",
        "Montgomery",
        "Nevada",
        "Newton",
        "Ouachita",
        "Perry",
        "Phillips",
        "Pike",
        "Poinsett",
        "Polk",
        "Pope",
        "Prairie",
        "Pulaski",
        "Randolph",
        "SaintFrancis",
        "Saline",
        "Scott",
        "Searcy",
        "Sebastian",
        "Sevier",
        "Sharp",
        "Stone",
        "Union",
        "VanBuren",
        "Washington",
        "White",
        "Woodruff",
        "Yell"
    ],
    "California": [
        "Alameda",
        "Alpine",
        "Amador",
        "Butte",
        "Calaveras",
        "Colusa",
        "ContraCosta",
        "DelNorte",
        "ElDorado",
        "Fresno",
        "Glenn",
        "Humboldt",
        "Imperial",
        "Inyo",
        "Kern",
        "Kings",
        "Lake",
        "Lassen",
        "LosAngeles",
        "Madera",
        "Marin",
        "Mariposa",
        "Mendocino",
        "Merced",
        "Modoc",
        "Mono",
        "Monterey",
        "Napa",
        "Nevada",
        "Orange",
        "Placer",
        "Plumas",
        "Riverside",
        "Sacramento",
        "SanBenito",
        "SanBernardino",
        "SanDiego",
        "SanFrancisco",
        "SanJoaquin",
        "SanLuisObispo",
        "SanMateo",
        "SantaBarbara",
        "SantaClara",
        "SantaCruz",
        "Shasta",
        "Sierra",
        "Siskiyou",
        "Solano",
        "Sonoma",
        "Stanislaus",
        "Sutter",
        "Tehama",
        "Trinity",
        "Tulare",
        "Tuolumne",
        "Ventura",
        "Yolo",
        "Yuba"
    ],
    "Colorado": [
        "Adams",
        "Alamosa",
        "Arapahoe",
        "Archuleta",
        "Baca",
        "Bent",
        "Boulder",
        "Broomfield",
        "Chaffee",
        "Cheyenne",
        "ClearCreek",
        "Conejos",
        "Costilla",
        "Crowley",
        "Custer",
        "Delta",
        "Denver",
        "Dolores",
        "Douglas",
        "Eagle",
        "ElPaso",
        "Elbert",
        "Fremont",
        "Garfield",
        "Gilpin",
        "Grand",
        "Gunnison",
        "Hinsdale",
        "Huerfano",
        "Jackson",
        "Jefferson",
        "Kiowa",
        "KitCarson",
        "LaPlata",
        "Lake",
        "Larimer",
        "LasAnimas",
        "Lincoln",
        "Logan",
        "Mesa",
        "Mineral",
        "Moffat",
        "Montezuma",
        "Montrose",
        "Morgan",
        "Otero",
        "Ouray",
        "Park",
        "Phillips",
        "Pitkin",
        "Prowers",
        "Pueblo",
        "RioBlanco",
        "RioGrande",
        "Routt",
        "Saguache",
        "SanJuan",
        "SanMiguel",
        "Sedgwick",
        "Summit",
        "Teller",
        "Washington",
        "Weld",
        "Yuma"
    ],
    "Connecticut": [
        "Fairfield",
        "Hartford",
        "Litchfield",
        "Middlesex",
        "NewHaven",
        "NewLondon",
        "Tolland",
        "Windham"
    ],
    "Delaware": [
        "Kent",
        "NewCastle",
        "Sussex"
    ],
    "DistrictofColumbia": [
        "DistrictofColumbia"
    ],
    "Florida": [
        "Alachua",
        "Baker",
        "Bay",
        "Bradford",
        "Brevard",
        "Broward",
        "Calhoun",
        "Charlotte",
        "Citrus",
        "Clay",
        "Collier",
        "Columbia",
        "Desoto",
        "Dixie",
        "Duval",
        "Escambia",
        "Flagler",
        "Franklin",
        "Gadsden",
        "Gilchrist",
        "Glades",
        "Gulf",
        "Hamilton",
        "Hardee",
        "Hendry",
        "Hernando",
        "Highlands",
        "Hillsborough",
        "Holmes",
        "IndianRiver",
        "Jackson",
        "Jefferson",
        "Lafayette",
        "Lake",
        "Lee",
        "Leon",
        "Levy",
        "Liberty",
        "Madison",
        "Manatee",
        "Marion",
        "Martin",
        "Miami-Dade",
        "Monroe",
        "Nassau",
        "Okaloosa",
        "Okeechobee",
        "Orange",
        "Osceola",
        "PalmBeach",
        "Pasco",
        "Pinellas",
        "Polk",
        "Putnam",
        "SaintJohns",
        "SaintLucie",
        "SantaRosa",
        "Sarasota",
        "Seminole",
        "Sumter",
        "Suwannee",
        "Taylor",
        "Union",
        "Volusia",
        "Wakulla",
        "Walton",
        "Washington"
    ],
    "Georgia": [
        "Appling",
        "Atkinson",
        "Bacon",
        "Baker",
        "Baldwin",
        "Banks",
        "Barrow",
        "Bartow",
        "BenHill",
        "Berrien",
        "Bibb",
        "Bleckley",
        "Brantley",
        "Brooks",
        "Bryan",
        "Bulloch",
        "Burke",
        "Butts",
        "Calhoun",
        "Camden",
        "Candler",
        "Carroll",
        "Catoosa",
        "Charlton",
        "Chatham",
        "Chattahoochee",
        "Chattooga",
        "Cherokee",
        "Clarke",
        "Clay",
        "Clayton",
        "Clinch",
        "Cobb",
        "Coffee",
        "Colquitt",
        "Columbia",
        "Cook",
        "Coweta",
        "Crawford",
        "Crisp",
        "Dade",
        "Dawson",
        "DeKalb",
        "Decatur",
        "Dodge",
        "Dooly",
        "Dougherty",
        "Douglas",
        "Early",
        "Echols",
        "Effingham",
        "Elbert",
        "Emanuel",
        "Evans",
        "Fannin",
        "Fayette",
        "Floyd",
        "Forsyth",
        "Franklin",
        "Fulton",
        "Gilmer",
        "Glascock",
        "Glynn",
        "Gordon",
        "Grady",
        "Greene",
        "Gwinnett",
        "Habersham",
        "Hall",
        "Hancock",
        "Haralson",
        "Harris",
        "Hart",
        "Heard",
        "Henry",
        "Houston",
        "Irwin",
        "Jackson",
        "Jasper",
        "JeffDavis",
        "Jefferson",
        "Jenkins",
        "Johnson",
        "Jones",
        "Lamar",
        "Lanier",
        "Laurens",
        "Lee",
        "Liberty",
        "Lincoln",
        "Long",
        "Lowndes",
        "Lumpkin",
        "Macon",
        "Madison",
        "Marion",
        "McDuffie",
        "McIntosh",
        "Meriwether",
        "Miller",
        "Mitchell",
        "Monroe",
        "Montgomery",
        "Morgan",
        "Murray",
        "Muscogee",
        "Newton",
        "Oconee",
        "Oglethorpe",
        "Paulding",
        "Peach",
        "Pickens",
        "Pierce",
        "Pike",
        "Polk",
        "Pulaski",
        "Putnam",
        "Quitman",
        "Rabun",
        "Randolph",
        "Richmond",
        "Rockdale",
        "Schley",
        "Screven",
        "Seminole",
        "Spalding",
        "Stephens",
        "Stewart",
        "Sumter",
        "Talbot",
        "Taliaferro",
        "Tattnall",
        "Taylor",
        "Telfair",
        "Terrell",
        "Thomas",
        "Tift",
        "Toombs",
        "Towns",
        "Treutlen",
        "Troup",
        "Turner",
        "Twiggs",
        "Union",
        "Upson",
        "Walker",
        "Walton",
        "Ware",
        "Warren",
        "Washington",
        "Wayne",
        "Webster",
        "Wheeler",
        "White",
        "Whitfield",
        "Wilcox",
        "Wilkes",
        "Wilkinson",
        "Worth"
    ],
    "Hawaii": [
        "Hawaii",
        "Honolulu",
        "Kalawao",
        "Kauai",
        "Maui"
    ],
    "Idaho": [
        "Ada",
        "Adams",
        "Bannock",
        "BearLake",
        "Benewah",
        "Bingham",
        "Blaine",
        "Boise",
        "Bonner",
        "Bonneville",
        "Boundary",
        "Butte",
        "Camas",
        "Canyon",
        "Caribou",
        "Cassia",
        "Clark",
        "Clearwater",
        "Custer",
        "Elmore",
        "Franklin",
        "Fremont",
        "Gem",
        "Gooding",
        "Idaho",
        "Jefferson",
        "Jerome",
        "Kootenai",
        "Latah",
        "Lemhi",
        "Lewis",
        "Lincoln",
        "Madison",
        "Minidoka",
        "NezPerce",
        "Oneida",
        "Owyhee",
        "Payette",
        "Power",
        "Shoshone",
        "Teton",
        "TwinFalls",
        "Valley",
        "Washington"
    ],
    "Illinois": [
        "Adams",
        "Alexander",
        "Bond",
        "Boone",
        "Brown",
        "Bureau",
        "Calhoun",
        "Carroll",
        "Cass",
        "Champaign",
        "Christian",
        "Clark",
        "Clay",
        "Clinton",
        "Coles",
        "Cook",
        "Crawford",
        "Cumberland",
        "DeKalb",
        "DeWitt",
        "Douglas",
        "Dupage",
        "Edgar",
        "Edwards",
        "Effingham",
        "Fayette",
        "Ford",
        "Franklin",
        "Fulton",
        "Gallatin",
        "Greene",
        "Grundy",
        "Hamilton",
        "Hancock",
        "Hardin",
        "Henderson",
        "Henry",
        "Iroquois",
        "Jackson",
        "Jasper",
        "Jefferson",
        "Jersey",
        "JoDaviess",
        "Johnson",
        "Kane",
        "Kankakee",
        "Kendall",
        "Knox",
        "LaSalle",
        "Lake",
        "LakeMichigan",
        "Lawrence",
        "Lee",
        "Livingston",
        "Logan",
        "Macon",
        "Macoupin",
        "Madison",
        "Marion",
        "Marshall",
        "Mason",
        "Massac",
        "McDonough",
        "McHenry",
        "McLean",
        "Menard",
        "Mercer",
        "Monroe",
        "Montgomery",
        "Morgan",
        "Moultrie",
        "Ogle",
        "Peoria",
        "Perry",
        "Piatt",
        "Pike",
        "Pope",
        "Pulaski",
        "Putnam",
        "Randolph",
        "Richland",
        "RockIsland",
        "SaintClair",
        "Saline",
        "Sangamon",
        "Schuyler",
        "Scott",
        "Shelby",
        "Stark",
        "Stephenson",
        "Tazewell",
        "Union",
        "Vermilion",
        "Wabash",
        "Warren",
        "Washington",
        "Wayne",
        "White",
        "Whiteside",
        "Will",
        "Williamson",
        "Winnebago",
        "Woodford"
    ],
    "Indiana": [
        "Adams",
        "Allen",
        "Bartholomew",
        "Benton",
        "Blackford",
        "Boone",
        "Brown",
        "Carroll",
        "Cass",
        "Clark",
        "Clay",
        "Clinton",
        "Crawford",
        "Daviess",
        "DeKalb",
        "Dearborn",
        "Decatur",
        "Delaware",
        "Dubois",
        "Elkhart",
        "Fayette",
        "Floyd",
        "Fountain",
        "Franklin",
        "Fulton",
        "Gibson",
        "Grant",
        "Greene",
        "Hamilton",
        "Hancock",
        "Harrison",
        "Hendricks",
        "Henry",
        "Howard",
        "Huntington",
        "Jackson",
        "Jasper",
        "Jay",
        "Jefferson",
        "Jennings",
        "Johnson",
        "Knox",
        "Kosciusko",
        "LaGrange",
        "LaPorte",
        "Lake",
        "LakeMichigan",
        "Lawrence",
        "Madison",
        "Marion",
        "Marshall",
        "Martin",
        "Miami",
        "Monroe",
        "Montgomery",
        "Morgan",
        "Newton",
        "Noble",
        "Ohio",
        "Orange",
        "Owen",
        "Parke",
        "Perry",
        "Pike",
        "Porter",
        "Posey",
        "Pulaski",
        "Putnam",
        "Randolph",
        "Ripley",
        "Rush",
        "SaintJoseph",
        "Scott",
        "Shelby",
        "Spencer",
        "Starke",
        "Steuben",
        "Sullivan",
        "Switzerland",
        "Tippecanoe",
        "Tipton",
        "Union",
        "Vanderburgh",
        "Vermillion",
        "Vigo",
        "Wabash",
        "Warren",
        "Warrick",
        "Washington",
        "Wayne",
        "Wells",
        "White",
        "Whitley"
    ],
    "Iowa": [
        "Adair",
        "Adams",
        "Allamakee",
        "Appanoose",
        "Audubon",
        "Benton",
        "BlackHawk",
        "Boone",
        "Bremer",
        "Buchanan",
        "BuenaVista",
        "Butler",
        "Calhoun",
        "Carroll",
        "Cass",
        "Cedar",
        "CerroGordo",
        "Cherokee",
        "Chickasaw",
        "Clarke",
        "Clay",
        "Clayton",
        "Clinton",
        "Crawford",
        "Dallas",
        "Davis",
        "Decatur",
        "Delaware",
        "DesMoines",
        "Dickinson",
        "Dubuque",
        "Emmet",
        "Fayette",
        "Floyd",
        "Franklin",
        "Fremont",
        "Greene",
        "Grundy",
        "Guthrie",
        "Hamilton",
        "Hancock",
        "Hardin",
        "Harrison",
        "Henry",
        "Howard",
        "Humboldt",
        "Ida",
        "Iowa",
        "Jackson",
        "Jasper",
        "Jefferson",
        "Johnson",
        "Jones",
        "Keokuk",
        "Kossuth",
        "Lee",
        "Linn",
        "Louisa",
        "Lucas",
        "Lyon",
        "Madison",
        "Mahaska",
        "Marion",
        "Marshall",
        "Mills",
        "Mitchell",
        "Monona",
        "Monroe",
        "Montgomery",
        "Muscatine",
        "O'Brien",
        "Osceola",
        "Page",
        "PaloAlto",
        "Plymouth",
        "Pocahontas",
        "Polk",
        "Pottawattamie",
        "Poweshiek",
        "Ringgold",
        "Sac",
        "Scott",
        "Shelby",
        "Sioux",
        "Story",
        "Tama",
        "Taylor",
        "Union",
        "VanBuren",
        "Wapello",
        "Warren",
        "Washington",
        "Wayne",
        "Webster",
        "Winnebago",
        "Winneshiek",
        "Woodbury",
        "Worth",
        "Wright"
    ],
    "Kansas": [
        "Allen",
        "Anderson",
        "Atchison",
        "Barber",
        "Barton",
        "Bourbon",
        "Brown",
        "Butler",
        "Chase",
        "Chautauqua",
        "Cherokee",
        "Cheyenne",
        "Clark",
        "Clay",
        "Cloud",
        "Coffey",
        "Comanche",
        "Cowley",
        "Crawford",
        "Decatur",
        "Dickinson",
        "Doniphan",
        "Douglas",
        "Edwards",
        "Elk",
        "Ellis",
        "Ellsworth",
        "Finney",
        "Ford",
        "Franklin",
        "Geary",
        "Gove",
        "Graham",
        "Grant",
        "Gray",
        "Greeley",
        "Greenwood",
        "Hamilton",
        "Harper",
        "Harvey",
        "Haskell",
        "Hodgeman",
        "Jackson",
        "Jefferson",
        "Jewell",
        "Johnson",
        "Kearny",
        "Kingman",
        "Kiowa",
        "Labette",
        "Lane",
        "Leavenworth",
        "Lincoln",
        "Linn",
        "Logan",
        "Lyon",
        "Marion",
        "Marshall",
        "McPherson",
        "Meade",
        "Miami",
        "Mitchell",
        "Montgomery",
        "Morris",
        "Morton",
        "Nemaha",
        "Neosho",
        "Ness",
        "Norton",
        "Osage",
        "Osborne",
        "Ottawa",
        "Pawnee",
        "Phillips",
        "Pottawatomie",
        "Pratt",
        "Rawlins",
        "Reno",
        "Republic",
        "Rice",
        "Riley",
        "Rooks",
        "Rush",
        "Russell",
        "Saline",
        "Scott",
        "Sedgwick",
        "Seward",
        "Shawnee",
        "Sheridan",
        "Sherman",
        "Smith",
        "Stafford",
        "Stanton",
        "Stevens",
        "Sumner",
        "Thomas",
        "Trego",
        "Wabaunsee",
        "Wallace",
        "Washington",
        "Wichita",
        "Wilson",
        "Woodson",
        "Wyandotte"
    ],
    "Kentucky": [
        "Adair",
        "Allen",
        "Anderson",
        "Ballard",
        "Barren",
        "Bath",
        "Bell",
        "Boone",
        "Bourbon",
        "Boyd",
        "Boyle",
        "Bracken",
        "Breathitt",
        "Breckinridge",
        "Bullitt",
        "Butler",
        "Caldwell",
        "Calloway",
        "Campbell",
        "Carlisle",
        "Carroll",
        "Carter",
        "Casey",
        "Christian",
        "Clark",
        "Clay",
        "Clinton",
        "Crittenden",
        "Cumberland",
        "Daviess",
        "Edmonson",
        "Elliott",
        "Estill",
        "Fayette",
        "Fleming",
        "Floyd",
        "Franklin",
        "Fulton",
        "Gallatin",
        "Garrard",
        "Grant",
        "Graves",
        "Grayson",
        "Green",
        "Greenup",
        "Hancock",
        "Hardin",
        "Harlan",
        "Harrison",
        "Hart",
        "Henderson",
        "Henry",
        "Hickman",
        "Hopkins",
        "Jackson",
        "Jefferson",
        "Jessamine",
        "Johnson",
        "Kenton",
        "Knott",
        "Knox",
        "Larue",
        "Laurel",
        "Lawrence",
        "Lee",
        "Leslie",
        "Letcher",
        "Lewis",
        "Lincoln",
        "Livingston",
        "Logan",
        "Lyon",
        "Madison",
        "Magoffin",
        "Marion",
        "Marshall",
        "Martin",
        "Mason",
        "McCracken",
        "McCreary",
        "McLean",
        "Meade",
        "Menifee",
        "Mercer",
        "Metcalfe",
        "Monroe",
        "Montgomery",
        "Morgan",
        "Muhlenberg",
        "Nelson",
        "Nicholas",
        "Ohio",
        "Oldham",
        "Owen",
        "Owsley",
        "Pendleton",
        "Perry",
        "Pike",
        "Powell",
        "Pulaski",
        "Robertson",
        "Rockcastle",
        "Rowan",
        "Russell",
        "Scott",
        "Shelby",
        "Simpson",
        "Spencer",
        "Taylor",
        "Todd",
        "Trigg",
        "Trimble",
        "Union",
        "Warren",
        "Washington",
        "Wayne",
        "Webster",
        "Whitley",
        "Wolfe",
        "Woodford"
    ],
    "Louisiana": [
        "Acadia",
        "Allen",
        "Ascension",
        "Assumption",
        "Avoyelles",
        "Beauregard",
        "Bienville",
        "Bossier",
        "Caddo",
        "Calcasieu",
        "Caldwell",
        "Cameron",
        "Catahoula",
        "Claiborne",
        "Concordia",
        "DeSoto",
        "EastBatonRouge",
        "EastCarroll",
        "EastFeliciana",
        "Evangeline",
        "Franklin",
        "Grant",
        "Iberia",
        "Iberville",
        "Jackson",
        "Jefferson",
        "JeffersonDavis",
        "LaSalle",
        "Lafayette",
        "Lafourche",
        "Lincoln",
        "Livingston",
        "Madison",
        "Morehouse",
        "Natchitoches",
        "Orleans",
        "Ouachita",
        "Plaquemines",
        "PointeCoupee",
        "Rapides",
        "RedRiver",
        "Richland",
        "Sabine",
        "SaintBernard",
        "SaintCharles",
        "SaintHelena",
        "SaintJames",
        "SaintJohntheBaptist",
        "SaintLandry",
        "SaintMartin",
        "SaintMary",
        "SaintTammany",
        "Tangipahoa",
        "Tensas",
        "Terrebonne",
        "Union",
        "Vermilion",
        "Vernon",
        "Washington",
        "Webster",
        "WestBatonRouge",
        "WestCarroll",
        "WestFeliciana",
        "Winn"
    ],
    "Maine": [
        "Androscoggin",
        "Aroostook",
        "Cumberland",
        "Franklin",
        "Hancock",
        "Kennebec",
        "Knox",
        "Lincoln",
        "Oxford",
        "Penobscot",
        "Piscataquis",
        "Sagadahoc",
        "Somerset",
        "Waldo",
        "Washington",
        "York"
    ],
    "Maryland": [
        "Allegany",
        "AnneArundel",
        "Baltimore",
        "Calvert",
        "Caroline",
        "Carroll",
        "Cecil",
        "Charles",
        "Dorchester",
        "Frederick",
        "Garrett",
        "Harford",
        "Howard",
        "Kent",
        "Montgomery",
        "PrinceGeorge's",
        "QueenAnne's",
        "SaintMary's",
        "Somerset",
        "Talbot",
        "Washington",
        "Wicomico",
        "Worcester"
    ],
    "Massachusetts": [
        "Barnstable",
        "Berkshire",
        "Bristol",
        "Dukes",
        "Essex",
        "Franklin",
        "Hampden",
        "Hampshire",
        "Middlesex",
        "Nantucket",
        "Norfolk",
        "Plymouth",
        "Suffolk",
        "Worcester"
    ],
    "Michigan": [
        "Alcona",
        "Alger",
        "Allegan",
        "Alpena",
        "Antrim",
        "Arenac",
        "Baraga",
        "Barry",
        "Bay",
        "Benzie",
        "Berrien",
        "Branch",
        "Calhoun",
        "Cass",
        "Charlevoix",
        "Cheboygan",
        "Chippewa",
        "Clare",
        "Clinton",
        "Crawford",
        "Delta",
        "Dickinson",
        "Eaton",
        "Emmet",
        "Genesee",
        "Gladwin",
        "Gogebic",
        "GrandTraverse",
        "Gratiot",
        "Hillsdale",
        "Houghton",
        "Huron",
        "Ingham",
        "Ionia",
        "Iosco",
        "Iron",
        "Isabella",
        "Jackson",
        "Kalamazoo",
        "Kalkaska",
        "Kent",
        "Keweenaw",
        "Lake",
        "LakeHurron",
        "LakeMichigan",
        "LakeSt.Clair",
        "LakeSuperior",
        "Lapeer",
        "Leelanau",
        "Lenawee",
        "Livingston",
        "Luce",
        "Mackinac",
        "Macomb",
        "Manistee",
        "Marquette",
        "Mason",
        "Mecosta",
        "Menominee",
        "Midland",
        "Missaukee",
        "Monroe",
        "Montcalm",
        "Montmorency",
        "Muskegon",
        "Newaygo",
        "Oakland",
        "Oceana",
        "Ogemaw",
        "Ontonagon",
        "Osceola",
        "Oscoda",
        "Otsego",
        "Ottawa",
        "PresqueIsle",
        "Roscommon",
        "Saginaw",
        "SaintClair",
        "SaintJoseph",
        "Sanilac",
        "Schoolcraft",
        "Shiawassee",
        "Tuscola",
        "VanBuren",
        "Washtenaw",
        "Wayne",
        "Wexford"
    ],
    "Minnesota": [
        "Aitkin",
        "Anoka",
        "Becker",
        "Beltrami",
        "Benton",
        "BigStone",
        "BlueEarth",
        "Brown",
        "Carlton",
        "Carver",
        "Cass",
        "Chippewa",
        "Chisago",
        "Clay",
        "Clearwater",
        "Cook",
        "Cottonwood",
        "CrowWing",
        "Dakota",
        "Dodge",
        "Douglas",
        "Faribault",
        "Fillmore",
        "Freeborn",
        "Goodhue",
        "Grant",
        "Hennepin",
        "Houston",
        "Hubbard",
        "Isanti",
        "Itasca",
        "Jackson",
        "Kanabec",
        "Kandiyohi",
        "Kittson",
        "Koochiching",
        "LacquiParle",
        "Lake",
        "LakeSuperior",
        "LakeoftheWoods",
        "LeSueur",
        "Lincoln",
        "Lyon",
        "Mahnomen",
        "Marshall",
        "Martin",
        "McLeod",
        "Meeker",
        "MilleLacs",
        "Morrison",
        "Mower",
        "Murray",
        "Nicollet",
        "Nobles",
        "Norman",
        "Olmsted",
        "OtterTail",
        "Pennington",
        "Pine",
        "Pipestone",
        "Polk",
        "Pope",
        "Ramsey",
        "RedLake",
        "Redwood",
        "Renville",
        "Rice",
        "Rock",
        "Roseau",
        "SaintLouis",
        "Scott",
        "Sherburne",
        "Sibley",
        "Stearns",
        "Steele",
        "Stevens",
        "Swift",
        "Todd",
        "Traverse",
        "Wabasha",
        "Wadena",
        "Waseca",
        "Washington",
        "Watonwan",
        "Wilkin",
        "Winona",
        "Wright",
        "YellowMedicine"
    ],
    "Mississippi": [
        "Adams",
        "Alcorn",
        "Amite",
        "Attala",
        "Benton",
        "Bolivar",
        "Calhoun",
        "Carroll",
        "Chickasaw",
        "Choctaw",
        "Claiborne",
        "Clarke",
        "Clay",
        "Coahoma",
        "Copiah",
        "Covington",
        "Desoto",
        "Forrest",
        "Franklin",
        "George",
        "Greene",
        "Grenada",
        "Hancock",
        "Harrison",
        "Hinds",
        "Holmes",
        "Humphreys",
        "Issaquena",
        "Itawamba",
        "Jackson",
        "Jasper",
        "Jefferson",
        "JeffersonDavis",
        "Jones",
        "Kemper",
        "Lafayette",
        "Lamar",
        "Lauderdale",
        "Lawrence",
        "Leake",
        "Lee",
        "Leflore",
        "Lincoln",
        "Lowndes",
        "Madison",
        "Marion",
        "Marshall",
        "Monroe",
        "Montgomery",
        "Neshoba",
        "Newton",
        "Noxubee",
        "Oktibbeha",
        "Panola",
        "PearlRiver",
        "Perry",
        "Pike",
        "Pontotoc",
        "Prentiss",
        "Quitman",
        "Rankin",
        "Scott",
        "Sharkey",
        "Simpson",
        "Smith",
        "Stone",
        "Sunflower",
        "Tallahatchie",
        "Tate",
        "Tippah",
        "Tishomingo",
        "Tunica",
        "Union",
        "Walthall",
        "Warren",
        "Washington",
        "Wayne",
        "Webster",
        "Wilkinson",
        "Winston",
        "Yalobusha",
        "Yazoo"
    ],
    "Missouri": [
        "Adair",
        "Andrew",
        "Atchison",
        "Audrain",
        "Barry",
        "Barton",
        "Bates",
        "Benton",
        "Bollinger",
        "Boone",
        "Buchanan",
        "Butler",
        "Caldwell",
        "Callaway",
        "Camden",
        "CapeGirardeau",
        "Carroll",
        "Carter",
        "Cass",
        "Cedar",
        "Chariton",
        "Christian",
        "Clark",
        "Clay",
        "Clinton",
        "Cole",
        "Cooper",
        "Crawford",
        "Dade",
        "Dallas",
        "Daviess",
        "DeKalb",
        "Dent",
        "Douglas",
        "Dunklin",
        "Franklin",
        "Gasconade",
        "Gentry",
        "Greene",
        "Grundy",
        "Harrison",
        "Henry",
        "Hickory",
        "Holt",
        "Howard",
        "Howell",
        "Iron",
        "Jackson",
        "Jasper",
        "Jefferson",
        "Johnson",
        "Knox",
        "Laclede",
        "Lafayette",
        "Lawrence",
        "Lewis",
        "Lincoln",
        "Linn",
        "Livingston",
        "Macon",
        "Madison",
        "Maries",
        "Marion",
        "McDonald",
        "Mercer",
        "Miller",
        "Mississippi",
        "Moniteau",
        "Monroe",
        "Montgomery",
        "Morgan",
        "NewMadrid",
        "Newton",
        "Nodaway",
        "Oregon",
        "Osage",
        "Ozark",
        "Pemiscot",
        "Perry",
        "Pettis",
        "Phelps",
        "Pike",
        "Platte",
        "Polk",
        "Pulaski",
        "Putnam",
        "Ralls",
        "Randolph",
        "Ray",
        "Reynolds",
        "Ripley",
        "SaintCharles",
        "SaintClair",
        "SaintFrancois",
        "SaintLouis",
        "SainteGenevieve",
        "Saline",
        "Schuyler",
        "Scotland",
        "Scott",
        "Shannon",
        "Shelby",
        "Stoddard",
        "Stone",
        "Sullivan",
        "Taney",
        "Texas",
        "Vernon",
        "Warren",
        "Washington",
        "Wayne",
        "Webster",
        "Worth",
        "Wright"
    ],
    "Montana": [
        "Beaverhead",
        "BigHorn",
        "Blaine",
        "Broadwater",
        "Carbon",
        "Carter",
        "Cascade",
        "Chouteau",
        "Custer",
        "Daniels",
        "Dawson",
        "DeerLodge",
        "Fallon",
        "Fergus",
        "Flathead",
        "Gallatin",
        "Garfield",
        "Glacier",
        "GoldenValley",
        "Granite",
        "Hill",
        "Jefferson",
        "JudithBasin",
        "Lake",
        "LewisandClark",
        "Liberty",
        "Lincoln",
        "Madison",
        "McCone",
        "Meagher",
        "Mineral",
        "Missoula",
        "Musselshell",
        "Park",
        "Petroleum",
        "Phillips",
        "Pondera",
        "PowderRiver",
        "Powell",
        "Prairie",
        "Ravalli",
        "Richland",
        "Roosevelt",
        "Rosebud",
        "Sanders",
        "Sheridan",
        "SilverBow",
        "Stillwater",
        "SweetGrass",
        "Teton",
        "Toole",
        "Treasure",
        "Valley",
        "Wheatland",
        "Wibaux",
        "Yellowstone"
    ],
    "Nebraska": [
        "Adams",
        "Antelope",
        "Arthur",
        "Banner",
        "Blaine",
        "Boone",
        "BoxButte",
        "Boyd",
        "Brown",
        "Buffalo",
        "Burt",
        "Butler",
        "Cass",
        "Cedar",
        "Chase",
        "Cherry",
        "Cheyenne",
        "Clay",
        "Colfax",
        "Cuming",
        "Custer",
        "Dakota",
        "Dawes",
        "Dawson",
        "Deuel",
        "Dixon",
        "Dodge",
        "Douglas",
        "Dundy",
        "Fillmore",
        "Franklin",
        "Frontier",
        "Furnas",
        "Gage",
        "Garden",
        "Garfield",
        "Gosper",
        "Grant",
        "Greeley",
        "Hall",
        "Hamilton",
        "Harlan",
        "Hayes",
        "Hitchcock",
        "Holt",
        "Hooker",
        "Howard",
        "Jefferson",
        "Johnson",
        "Kearney",
        "Keith",
        "KeyaPaha",
        "Kimball",
        "Knox",
        "Lancaster",
        "Lincoln",
        "Logan",
        "Loup",
        "Madison",
        "McPherson",
        "Merrick",
        "Morrill",
        "Nance",
        "Nemaha",
        "Nuckolls",
        "Otoe",
        "Pawnee",
        "Perkins",
        "Phelps",
        "Pierce",
        "Platte",
        "Polk",
        "RedWillow",
        "Richardson",
        "Rock",
        "Saline",
        "Sarpy",
        "Saunders",
        "ScottsBluff",
        "Seward",
        "Sheridan",
        "Sherman",
        "Sioux",
        "Stanton",
        "Thayer",
        "Thomas",
        "Thurston",
        "Valley",
        "Washington",
        "Wayne",
        "Webster",
        "Wheeler",
        "York"
    ],
    "Nevada": [
        "CarsonCity",
        "Churchill",
        "Clark",
        "Douglas",
        "Elko",
        "Esmeralda",
        "Eureka",
        "Humboldt",
        "Lander",
        "Lincoln",
        "Lyon",
        "Mineral",
        "Nye",
        "Pershing",
        "Storey",
        "Washoe",
        "WhitePine"
    ],
    "NewHampshire": [
        "Belknap",
        "Carroll",
        "Cheshire",
        "Coos",
        "Grafton",
        "Hillsborough",
        "Merrimack",
        "Rockingham",
        "Strafford",
        "Sullivan"
    ],
    "NewJersey": [
        "Atlantic",
        "Bergen",
        "Burlington",
        "Camden",
        "CapeMay",
        "Cumberland",
        "Essex",
        "Gloucester",
        "Hudson",
        "Hunterdon",
        "Mercer",
        "Middlesex",
        "Monmouth",
        "Morris",
        "Ocean",
        "Passaic",
        "Salem",
        "Somerset",
        "Sussex",
        "Union",
        "Warren"
    ],
    "NewMexico": [
        "Bernalillo",
        "Catron",
        "Chaves",
        "Cibola",
        "Colfax",
        "Curry",
        "Debaca",
        "DonaAna",
        "Eddy",
        "Grant",
        "Guadalupe",
        "Harding",
        "Hidalgo",
        "Lea",
        "Lincoln",
        "LosAlamos",
        "Luna",
        "McKinley",
        "Mora",
        "Otero",
        "Quay",
        "RioArriba",
        "Roosevelt",
        "SanJuan",
        "SanMiguel",
        "Sandoval",
        "SantaFe",
        "Sierra",
        "Socorro",
        "Taos",
        "Torrance",
        "Union",
        "Valencia"
    ],
    "NewYork": [
        "Albany",
        "Allegany",
        "Bronx",
        "Broome",
        "Cattaraugus",
        "Cayuga",
        "Chautauqua",
        "Chemung",
        "Chenango",
        "Clinton",
        "Columbia",
        "Cortland",
        "Delaware",
        "Dutchess",
        "Erie",
        "Essex",
        "Franklin",
        "Fulton",
        "Genesee",
        "Greene",
        "Hamilton",
        "Herkimer",
        "Jefferson",
        "Kings",
        "LakeOntario",
        "Lewis",
        "Livingston",
        "Madison",
        "Monroe",
        "Montgomery",
        "Nassau",
        "NewYork",
        "Niagara",
        "Oneida",
        "Onondaga",
        "Ontario",
        "Orange",
        "Orleans",
        "Oswego",
        "Otsego",
        "Putnam",
        "Queens",
        "Rensselaer",
        "Richmond",
        "Rockland",
        "SaintLawrence",
        "Saratoga",
        "Schenectady",
        "Schoharie",
        "Schuyler",
        "Seneca",
        "Steuben",
        "Suffolk",
        "Sullivan",
        "Tioga",
        "Tompkins",
        "Ulster",
        "Warren",
        "Washington",
        "Wayne",
        "Westchester",
        "Wyoming",
        "Yates"
    ],
    "NorthCarolina": [
        "Alamance",
        "Alexander",
        "Alleghany",
        "Anson",
        "Ashe",
        "Avery",
        "Beaufort",
        "Bertie",
        "Bladen",
        "Brunswick",
        "Buncombe",
        "Burke",
        "Cabarrus",
        "Caldwell",
        "Camden",
        "Carteret",
        "Caswell",
        "Catawba",
        "Chatham",
        "Cherokee",
        "Chowan",
        "Clay",
        "Cleveland",
        "Columbus",
        "Craven",
        "Cumberland",
        "Currituck",
        "Dare",
        "Davidson",
        "Davie",
        "Duplin",
        "Durham",
        "Edgecombe",
        "Forsyth",
        "Franklin",
        "Gaston",
        "Gates",
        "Graham",
        "Granville",
        "Greene",
        "Guilford",
        "Halifax",
        "Harnett",
        "Haywood",
        "Henderson",
        "Hertford",
        "Hoke",
        "Hyde",
        "Iredell",
        "Jackson",
        "Johnston",
        "Jones",
        "Lee",
        "Lenoir",
        "Lincoln",
        "Macon",
        "Madison",
        "Martin",
        "McDowell",
        "Mecklenburg",
        "Mitchell",
        "Montgomery",
        "Moore",
        "Nash",
        "NewHanover",
        "Northampton",
        "Onslow",
        "Orange",
        "Pamlico",
        "Pasquotank",
        "Pender",
        "Perquimans",
        "Person",
        "Pitt",
        "Polk",
        "Randolph",
        "Richmond",
        "Robeson",
        "Rockingham",
        "Rowan",
        "Rutherford",
        "Sampson",
        "Scotland",
        "Stanly",
        "Stokes",
        "Surry",
        "Swain",
        "Transylvania",
        "Tyrrell",
        "Union",
        "Vance",
        "Wake",
        "Warren",
        "Washington",
        "Watauga",
        "Wayne",
        "Wilkes",
        "Wilson",
        "Yadkin",
        "Yancey"
    ],
    "NorthDakota": [
        "Adams",
        "Barnes",
        "Benson",
        "Billings",
        "Bottineau",
        "Bowman",
        "Burke",
        "Burleigh",
        "Cass",
        "Cavalier",
        "Dickey",
        "Divide",
        "Dunn",
        "Eddy",
        "Emmons",
        "Foster",
        "GoldenValley",
        "GrandForks",
        "Grant",
        "Griggs",
        "Hettinger",
        "Kidder",
        "Lamoure",
        "Logan",
        "McHenry",
        "McIntosh",
        "McKenzie",
        "McLean",
        "Mercer",
        "Morton",
        "Mountrail",
        "Nelson",
        "Oliver",
        "Pembina",
        "Pierce",
        "Ramsey",
        "Ransom",
        "Renville",
        "Richland",
        "Rolette",
        "Sargent",
        "Sheridan",
        "Sioux",
        "Slope",
        "Stark",
        "Steele",
        "Stutsman",
        "Towner",
        "Traill",
        "Walsh",
        "Ward",
        "Wells",
        "Williams"
    ],
    "Ohio": [
        "Adams",
        "Allen",
        "Ashland",
        "Ashtabula",
        "Athens",
        "Auglaize",
        "Belmont",
        "Brown",
        "Butler",
        "Carroll",
        "Champaign",
        "Clark",
        "Clermont",
        "Clinton",
        "Columbiana",
        "Coshocton",
        "Crawford",
        "Cuyahoga",
        "Darke",
        "Defiance",
        "Delaware",
        "Erie",
        "Fairfield",
        "Fayette",
        "Franklin",
        "Fulton",
        "Gallia",
        "Geauga",
        "Greene",
        "Guernsey",
        "Hamilton",
        "Hancock",
        "Hardin",
        "Harrison",
        "Henry",
        "Highland",
        "Hocking",
        "Holmes",
        "Huron",
        "Jackson",
        "Jefferson",
        "Knox",
        "Lake",
        "LakeErie",
        "Lawrence",
        "Licking",
        "Logan",
        "Lorain",
        "Lucas",
        "Madison",
        "Mahoning",
        "Marion",
        "Medina",
        "Meigs",
        "Mercer",
        "Miami",
        "Monroe",
        "Montgomery",
        "Morgan",
        "Morrow",
        "Muskingum",
        "Noble",
        "Ottawa",
        "Paulding",
        "Perry",
        "Pickaway",
        "Pike",
        "Portage",
        "Preble",
        "Putnam",
        "Richland",
        "Ross",
        "Sandusky",
        "Scioto",
        "Seneca",
        "Shelby",
        "Stark",
        "Summit",
        "Trumbull",
        "Tuscarawas",
        "Union",
        "VanWert",
        "Vinton",
        "Warren",
        "Washington",
        "Wayne",
        "Williams",
        "Wood",
        "Wyandot"
    ],
    "Oklahoma": [
        "Adair",
        "Alfalfa",
        "Atoka",
        "Beaver",
        "Beckham",
        "Blaine",
        "Bryan",
        "Caddo",
        "Canadian",
        "Carter",
        "Cherokee",
        "Choctaw",
        "Cimarron",
        "Cleveland",
        "Coal",
        "Comanche",
        "Cotton",
        "Craig",
        "Creek",
        "Custer",
        "Delaware",
        "Dewey",
        "Ellis",
        "Garfield",
        "Garvin",
        "Grady",
        "Grant",
        "Greer",
        "Harmon",
        "Harper",
        "Haskell",
        "Hughes",
        "Jackson",
        "Jefferson",
        "Johnston",
        "Kay",
        "Kingfisher",
        "Kiowa",
        "Latimer",
        "LeFlore",
        "Lincoln",
        "Logan",
        "Love",
        "Major",
        "Marshall",
        "Mayes",
        "McClain",
        "McCurtain",
        "McIntosh",
        "Murray",
        "Muskogee",
        "Noble",
        "Nowata",
        "Okfuskee",
        "Oklahoma",
        "Okmulgee",
        "Osage",
        "Ottawa",
        "Pawnee",
        "Payne",
        "Pittsburg",
        "Pontotoc",
        "Pottawatomie",
        "Pushmataha",
        "RogerMills",
        "Rogers",
        "Seminole",
        "Sequoyah",
        "Stephens",
        "Texas",
        "Tillman",
        "Tulsa",
        "Wagoner",
        "Washington",
        "Washita",
        "Woods",
        "Woodward"
    ],
    "Oregon": [
        "Baker",
        "Benton",
        "Clackamas",
        "Clatsop",
        "Columbia",
        "Coos",
        "Crook",
        "Curry",
        "Deschutes",
        "Douglas",
        "Gilliam",
        "Grant",
        "Harney",
        "HoodRiver",
        "Jackson",
        "Jefferson",
        "Josephine",
        "Klamath",
        "Lake",
        "Lane",
        "Lincoln",
        "Linn",
        "Malheur",
        "Marion",
        "Morrow",
        "Multnomah",
        "Polk",
        "Sherman",
        "Tillamook",
        "Umatilla",
        "Union",
        "Wallowa",
        "Wasco",
        "Washington",
        "Wheeler",
        "Yamhill"
    ],
    "Pennsylvania": [
        "Adams",
        "Allegheny",
        "Armstrong",
        "Beaver",
        "Bedford",
        "Berks",
        "Blair",
        "Bradford",
        "Bucks",
        "Butler",
        "Cambria",
        "Cameron",
        "Carbon",
        "Centre",
        "Chester",
        "Clarion",
        "Clearfield",
        "Clinton",
        "Columbia",
        "Crawford",
        "Cumberland",
        "Dauphin",
        "Delaware",
        "Elk",
        "Erie",
        "Fayette",
        "Forest",
        "Franklin",
        "Fulton",
        "Greene",
        "Huntingdon",
        "Indiana",
        "Jefferson",
        "Juniata",
        "Lackawanna",
        "Lancaster",
        "Lawrence",
        "Lebanon",
        "Lehigh",
        "Luzerne",
        "Lycoming",
        "McKean",
        "Mercer",
        "Mifflin",
        "Monroe",
        "Montgomery",
        "Montour",
        "Northampton",
        "Northumberland",
        "Perry",
        "Philadelphia",
        "Pike",
        "Potter",
        "Schuylkill",
        "Snyder",
        "Somerset",
        "Sullivan",
        "Susquehanna",
        "Tioga",
        "Union",
        "Venango",
        "Warren",
        "Washington",
        "Wayne",
        "Westmoreland",
        "Wyoming",
        "York"
    ],
    "RhodeIsland": [
        "Bristol",
        "Kent",
        "Newport",
        "Providence",
        "Washington"
    ],
    "SouthCarolina": [
        "Abbeville",
        "Aiken",
        "Allendale",
        "Anderson",
        "Bamberg",
        "Barnwell",
        "Beaufort",
        "Berkeley",
        "Calhoun",
        "Charleston",
        "Cherokee",
        "Chester",
        "Chesterfield",
        "Clarendon",
        "Colleton",
        "Darlington",
        "Dillon",
        "Dorchester",
        "Edgefield",
        "Fairfield",
        "Florence",
        "Georgetown",
        "Greenville",
        "Greenwood",
        "Hampton",
        "Horry",
        "Jasper",
        "Kershaw",
        "Lancaster",
        "Laurens",
        "Lee",
        "Lexington",
        "Marion",
        "Marlboro",
        "McCormick",
        "Newberry",
        "Oconee",
        "Orangeburg",
        "Pickens",
        "Richland",
        "Saluda",
        "Spartanburg",
        "Sumter",
        "Union",
        "Williamsburg",
        "York"
    ],
    "SouthDakota": [
        "Aurora",
        "Beadle",
        "Bennett",
        "BonHomme",
        "Brookings",
        "Brown",
        "Brule",
        "Buffalo",
        "Butte",
        "Campbell",
        "CharlesMix",
        "Clark",
        "Clay",
        "Codington",
        "Corson",
        "Custer",
        "Davison",
        "Day",
        "Deuel",
        "Dewey",
        "Douglas",
        "Edmunds",
        "FallRiver",
        "Faulk",
        "Grant",
        "Gregory",
        "Haakon",
        "Hamlin",
        "Hand",
        "Hanson",
        "Harding",
        "Hughes",
        "Hutchinson",
        "Hyde",
        "Jackson",
        "Jerauld",
        "Jones",
        "Kingsbury",
        "Lake",
        "Lawrence",
        "Lincoln",
        "Lyman",
        "Marshall",
        "McCook",
        "McPherson",
        "Meade",
        "Mellette",
        "Miner",
        "Minnehaha",
        "Moody",
        "Pennington",
        "Perkins",
        "Potter",
        "Roberts",
        "Sanborn",
        "Shannon",
        "Spink",
        "Stanley",
        "Sully",
        "Todd",
        "Tripp",
        "Turner",
        "Union",
        "Walworth",
        "Yankton",
        "Ziebach"
    ],
    "Tennessee": [
        "Anderson",
        "Bedford",
        "Benton",
        "Bledsoe",
        "Blount",
        "Bradley",
        "Campbell",
        "Cannon",
        "Carroll",
        "Carter",
        "Cheatham",
        "Chester",
        "Claiborne",
        "Clay",
        "Cocke",
        "Coffee",
        "Crockett",
        "Cumberland",
        "Davidson",
        "DeKalb",
        "Decatur",
        "Dickson",
        "Dyer",
        "Fayette",
        "Fentress",
        "Franklin",
        "Gibson",
        "Giles",
        "Grainger",
        "Greene",
        "Grundy",
        "Hamblen",
        "Hamilton",
        "Hancock",
        "Hardeman",
        "Hardin",
        "Hawkins",
        "Haywood",
        "Henderson",
        "Henry",
        "Hickman",
        "Houston",
        "Humphreys",
        "Jackson",
        "Jefferson",
        "Johnson",
        "Knox",
        "Lake",
        "Lauderdale",
        "Lawrence",
        "Lewis",
        "Lincoln",
        "Loudon",
        "Macon",
        "Madison",
        "Marion",
        "Marshall",
        "Maury",
        "McMinn",
        "McNairy",
        "Meigs",
        "Monroe",
        "Montgomery",
        "Moore",
        "Morgan",
        "Obion",
        "Overton",
        "Perry",
        "Pickett",
        "Polk",
        "Putnam",
        "Rhea",
        "Roane",
        "Robertson",
        "Rutherford",
        "Scott",
        "Sequatchie",
        "Sevier",
        "Shelby",
        "Smith",
        "Stewart",
        "Sullivan",
        "Sumner",
        "Tipton",
        "Trousdale",
        "Unicoi",
        "Union",
        "VanBuren",
        "Warren",
        "Washington",
        "Wayne",
        "Weakley",
        "White",
        "Williamson",
        "Wilson"
    ],
    "Texas": [
        "Anderson",
        "Andrews",
        "Angelina",
        "Aransas",
        "Archer",
        "Armstrong",
        "Atascosa",
        "Austin",
        "Bailey",
        "Bandera",
        "Bastrop",
        "Baylor",
        "Bee",
        "Bell",
        "Bexar",
        "Blanco",
        "Borden",
        "Bosque",
        "Bowie",
        "Brazoria",
        "Brazos",
        "Brewster",
        "Briscoe",
        "Brooks",
        "Brown",
        "Burleson",
        "Burnet",
        "Caldwell",
        "Calhoun",
        "Callahan",
        "Cameron",
        "Camp",
        "Carson",
        "Cass",
        "Castro",
        "Chambers",
        "Cherokee",
        "Childress",
        "Clay",
        "Cochran",
        "Coke",
        "Coleman",
        "Collin",
        "Collingsworth",
        "Colorado",
        "Comal",
        "Comanche",
        "Concho",
        "Cooke",
        "Coryell",
        "Cottle",
        "Crane",
        "Crockett",
        "Crosby",
        "Culberson",
        "Dallam",
        "Dallas",
        "Dawson",
        "DeafSmith",
        "Delta",
        "Denton",
        "Dewitt",
        "Dickens",
        "Dimmit",
        "Donley",
        "Duval",
        "Eastland",
        "Ector",
        "Edwards",
        "ElPaso",
        "Ellis",
        "Erath",
        "Falls",
        "Fannin",
        "Fayette",
        "Fisher",
        "Floyd",
        "Foard",
        "FortBend",
        "Franklin",
        "Freestone",
        "Frio",
        "Gaines",
        "Galveston",
        "Garza",
        "Gillespie",
        "Glasscock",
        "Goliad",
        "Gonzales",
        "Gray",
        "Grayson",
        "Gregg",
        "Grimes",
        "Guadalupe",
        "Hale",
        "Hall",
        "Hamilton",
        "Hansford",
        "Hardeman",
        "Hardin",
        "Harris",
        "Harrison",
        "Hartley",
        "Haskell",
        "Hays",
        "Hemphill",
        "Henderson",
        "Hidalgo",
        "Hill",
        "Hockley",
        "Hood",
        "Hopkins",
        "Houston",
        "Howard",
        "Hudspeth",
        "Hunt",
        "Hutchinson",
        "Irion",
        "Jack",
        "Jackson",
        "Jasper",
        "JeffDavis",
        "Jefferson",
        "JimHogg",
        "JimWells",
        "Johnson",
        "Jones",
        "Karnes",
        "Kaufman",
        "Kendall",
        "Kenedy",
        "Kent",
        "Kerr",
        "Kimble",
        "King",
        "Kinney",
        "Kleberg",
        "Knox",
        "LaSalle",
        "Lamar",
        "Lamb",
        "Lampasas",
        "Lavaca",
        "Lee",
        "Leon",
        "Liberty",
        "Limestone",
        "Lipscomb",
        "LiveOak",
        "Llano",
        "Loving",
        "Lubbock",
        "Lynn",
        "Madison",
        "Marion",
        "Martin",
        "Mason",
        "Matagorda",
        "Maverick",
        "McCulloch",
        "McLennan",
        "McMullen",
        "Medina",
        "Menard",
        "Midland",
        "Milam",
        "Mills",
        "Mitchell",
        "Montague",
        "Montgomery",
        "Moore",
        "Morris",
        "Motley",
        "Nacogdoches",
        "Navarro",
        "Newton",
        "Nolan",
        "Nueces",
        "Ochiltree",
        "Oldham",
        "Orange",
        "PaloPinto",
        "Panola",
        "Parker",
        "Parmer",
        "Pecos",
        "Polk",
        "Potter",
        "Presidio",
        "Rains",
        "Randall",
        "Reagan",
        "Real",
        "RedRiver",
        "Reeves",
        "Refugio",
        "Roberts",
        "Robertson",
        "Rockwall",
        "Runnels",
        "Rusk",
        "Sabine",
        "SanAugustine",
        "SanJacinto",
        "SanPatricio",
        "SanSaba",
        "Schleicher",
        "Scurry",
        "Shackelford",
        "Shelby",
        "Sherman",
        "Smith",
        "Somervell",
        "Starr",
        "Stephens",
        "Sterling",
        "Stonewall",
        "Sutton",
        "Swisher",
        "Tarrant",
        "Taylor",
        "Terrell",
        "Terry",
        "Throckmorton",
        "Titus",
        "TomGreen",
        "Travis",
        "Trinity",
        "Tyler",
        "Upshur",
        "Upton",
        "Uvalde",
        "ValVerde",
        "VanZandt",
        "Victoria",
        "Walker",
        "Waller",
        "Ward",
        "Washington",
        "Webb",
        "Wharton",
        "Wheeler",
        "Wichita",
        "Wilbarger",
        "Willacy",
        "Williamson",
        "Wilson",
        "Winkler",
        "Wise",
        "Wood",
        "Yoakum",
        "Young",
        "Zapata",
        "Zavala"
    ],
    "Utah": [
        "Beaver",
        "BoxElder",
        "Cache",
        "Carbon",
        "Daggett",
        "Davis",
        "Duchesne",
        "Emery",
        "Garfield",
        "Grand",
        "Iron",
        "Juab",
        "Kane",
        "Millard",
        "Morgan",
        "Piute",
        "Rich",
        "SaltLake",
        "SanJuan",
        "Sanpete",
        "Sevier",
        "Summit",
        "Tooele",
        "Uintah",
        "Utah",
        "Wasatch",
        "Washington",
        "Wayne",
        "Weber"
    ],
    "Vermont": [
        "Addison",
        "Bennington",
        "Caledonia",
        "Chittenden",
        "Essex",
        "Franklin",
        "GrandIsle",
        "Lamoille",
        "Orange",
        "Orleans",
        "Rutland",
        "Washington",
        "Windham",
        "Windsor"
    ],
    "Virginia": [
        "Accomack",
        "Albemarle",
        "Alexandria",
        "Alleghany",
        "Amelia",
        "Amherst",
        "Appomattox",
        "Arlington",
        "Augusta",
        "Bath",
        "Bedford",
        "BedfordCity",
        "Bland",
        "Botetourt",
        "Bristol",
        "Brunswick",
        "Buchanan",
        "Buckingham",
        "BuenaVista",
        "Campbell",
        "Caroline",
        "Carroll",
        "CharlesCity",
        "Charlotte",
        "Charlottesville",
        "Chesapeake",
        "Chesterfield",
        "Clarke",
        "CliftonForgeCity",
        "ColonialHeights",
        "Covington",
        "Craig",
        "Culpeper",
        "Cumberland",
        "Danville",
        "Dickenson",
        "Dinwiddie",
        "Emporia",
        "Essex",
        "Fairfax",
        "FairfaxCity",
        "FallsChurch",
        "Fauquier",
        "Floyd",
        "Fluvanna",
        "Franklin",
        "Frederick",
        "Fredericksburg",
        "Galax",
        "Giles",
        "Gloucester",
        "Goochland",
        "Grayson",
        "Greene",
        "Greensville",
        "Halifax",
        "Hampton",
        "Hanover",
        "Harrisonburg",
        "Henrico",
        "Henry",
        "Highland",
        "Hopewell",
        "IsleofWight",
        "JamesCity",
        "KingGeorge",
        "KingWilliam",
        "KingandQueen",
        "Lancaster",
        "Lee",
        "Lexington",
        "Loudoun",
        "Louisa",
        "Lunenburg",
        "Lynchburg",
        "Madison",
        "Manassas",
        "ManassasPark",
        "Martinsville",
        "Mathews",
        "Mecklenburg",
        "Middlesex",
        "Montgomery",
        "Nelson",
        "NewKent",
        "NewportNews",
        "Norfolk",
        "Northampton",
        "Northumberland",
        "Norton",
        "Nottoway",
        "Orange",
        "Page",
        "Patrick",
        "Petersburg",
        "Pittsylvania",
        "Poquoson",
        "Portsmouth",
        "Powhatan",
        "PrinceEdward",
        "PrinceGeorge",
        "PrinceWilliam",
        "Pulaski",
        "Radford",
        "Rappahannock",
        "Richmond",
        "Roanoke",
        "RoanokeCity",
        "Rockbridge",
        "Rockingham",
        "Russell",
        "Salem",
        "Scott",
        "Shenandoah",
        "Smyth",
        "Southampton",
        "Spotsylvania",
        "Stafford",
        "Staunton",
        "Suffolk",
        "Surry",
        "Sussex",
        "Tazewell",
        "VirginiaBeach",
        "Warren",
        "Washington",
        "Waynesboro",
        "Westmoreland",
        "Williamsburg",
        "Winchester",
        "Wise",
        "Wythe",
        "York"
    ],
    "Washington": [
        "Adams",
        "Asotin",
        "Benton",
        "Chelan",
        "Clallam",
        "Clark",
        "Columbia",
        "Cowlitz",
        "Douglas",
        "Ferry",
        "Franklin",
        "Garfield",
        "Grant",
        "GraysHarbor",
        "Island",
        "Jefferson",
        "King",
        "Kitsap",
        "Kittitas",
        "Klickitat",
        "Lewis",
        "Lincoln",
        "Mason",
        "Okanogan",
        "Pacific",
        "PendOreille",
        "Pierce",
        "SanJuan",
        "Skagit",
        "Skamania",
        "Snohomish",
        "Spokane",
        "Stevens",
        "Thurston",
        "Wahkiakum",
        "WallaWalla",
        "Whatcom",
        "Whitman",
        "Yakima"
    ],
    "WestVirginia": [
        "Barbour",
        "Berkeley",
        "Boone",
        "Braxton",
        "Brooke",
        "Cabell",
        "Calhoun",
        "Clay",
        "Doddridge",
        "Fayette",
        "Gilmer",
        "Grant",
        "Greenbrier",
        "Hampshire",
        "Hancock",
        "Hardy",
        "Harrison",
        "Jackson",
        "Jefferson",
        "Kanawha",
        "Lewis",
        "Lincoln",
        "Logan",
        "Marion",
        "Marshall",
        "Mason",
        "McDowell",
        "Mercer",
        "Mineral",
        "Mingo",
        "Monongalia",
        "Monroe",
        "Morgan",
        "Nicholas",
        "Ohio",
        "Pendleton",
        "Pleasants",
        "Pocahontas",
        "Preston",
        "Putnam",
        "Raleigh",
        "Randolph",
        "Ritchie",
        "Roane",
        "Summers",
        "Taylor",
        "Tucker",
        "Tyler",
        "Upshur",
        "Wayne",
        "Webster",
        "Wetzel",
        "Wirt",
        "Wood",
        "Wyoming"
    ],
    "Wisconsin": [
        "Adams",
        "Ashland",
        "Barron",
        "Bayfield",
        "Brown",
        "Buffalo",
        "Burnett",
        "Calumet",
        "Chippewa",
        "Clark",
        "Columbia",
        "Crawford",
        "Dane",
        "Dodge",
        "Door",
        "Douglas",
        "Dunn",
        "EauClaire",
        "Florence",
        "FondduLac",
        "Forest",
        "Grant",
        "Green",
        "GreenLake",
        "Iowa",
        "Iron",
        "Jackson",
        "Jefferson",
        "Juneau",
        "Kenosha",
        "Kewaunee",
        "LaCrosse",
        "Lafayette",
        "LakeMichigan",
        "LakeSuperior",
        "Langlade",
        "Lincoln",
        "Manitowoc",
        "Marathon",
        "Marinette",
        "Marquette",
        "Menominee",
        "Milwaukee",
        "Monroe",
        "Oconto",
        "Oneida",
        "Outagamie",
        "Ozaukee",
        "Pepin",
        "Pierce",
        "Polk",
        "Portage",
        "Price",
        "Racine",
        "Richland",
        "Rock",
        "Rusk",
        "SaintCroix",
        "Sauk",
        "Sawyer",
        "Shawano",
        "Sheboygan",
        "Taylor",
        "Trempealeau",
        "Vernon",
        "Vilas",
        "Walworth",
        "Washburn",
        "Washington",
        "Waukesha",
        "Waupaca",
        "Waushara",
        "Winnebago",
        "Wood"
    ],
    "Wyoming": [
        "Albany",
        "BigHorn",
        "Campbell",
        "Carbon",
        "Converse",
        "Crook",
        "Fremont",
        "Goshen",
        "HotSprings",
        "Johnson",
        "Laramie",
        "Lincoln",
        "Natrona",
        "Niobrara",
        "Park",
        "Platte",
        "Sheridan",
        "Sublette",
        "Sweetwater",
        "Teton",
        "Uinta",
        "Washakie",
        "Weston"
    ]
}


### Cell 6 and 7

In [ ]:
import os
import time
from datetime import timedelta
import pandas as pd
from tabulate import tabulate
import geopandas as gpd
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import numpy as np

# Cell 6
# Generating maps by county for each state
print("\nStarting to generate maps by county for each state...\n")

# Load the US counties dataset from the provided GeoJSON file
try:
    counties = gpd.read_file("/content/drive/MyDrive/CI_Sentinal/old/datasets/raw/geographic/gadm41_USA_2.json")
    if counties.crs != "EPSG:4326":
        counties = counties.to_crs("EPSG:4326")
    print("US counties boundaries loaded successfully.")
except Exception as e:
    print(f"Error loading US counties boundaries: {e}")

# Get unique state names from the loaded counties dataset
# Note: Add a space between words if they are concatenated in the GeoJSON
counties['NAME_1'] = counties['NAME_1'].apply(lambda x: ' '.join(x.split()).title())
counties['NAME_2'] = counties['NAME_2'].apply(lambda x: ' '.join(x.split()).title())
states_in_dataset = counties['NAME_1'].unique()

# Total number of counties for progress tracking
total_counties = len(counties)
processed_counties = 0

# Loop through each state in the user-defined states_counties list
for state_name, counties_list in states_counties.items():
    state_name = state_name.title()  # Capitalize state name properly

    if state_name not in states_in_dataset:
        print(f"State {state_name} is not in the dataset. Skipping...")
        continue

    print(f"\nProcessing state: {state_name}...\n")

    # Create a GeoDataFrame for the current state
    state_gdf = counties[counties['NAME_1'] == state_name]

    # Create a directory for the state if it doesn't exist
    state_dir = f"/content/drive/MyDrive/CI_Sentinal/geoJSON/maps/state_maps/{state_name.replace(' ', '_')}"
    os.makedirs(state_dir, exist_ok=True)

    # Lists to accumulate state-wide data for later use in Cell 7
    all_state_facilities = pd.DataFrame()
    weighted_coords_state = []

    # Initialize state-level start time for time tracking
    state_start_time = time.time()
    processed_counties_in_state = 0

    # Now, process each county within the state, as defined in states_counties
    for county_name in counties_list:
        county_name = county_name.title()  # Capitalize county name properly

        print(f"\nProcessing county: {county_name} in {state_name}...\n")
        county_start_time = time.time()

        # Create a GeoDataFrame for the current county
        county_gdf = state_gdf[state_gdf['NAME_2'] == county_name]

        # If the county doesn't exist in the dataset, skip it
        if county_gdf.empty:
            print(f"County {county_name} not found in {state_name}. Skipping...\n")
            continue

        # Create a directory for the county if it doesn't exist
        county_dir = f"{state_dir}/{county_name.replace(' ', '_')}"
        os.makedirs(county_dir, exist_ok=True)

        # Data tracking
        datasets_present = []
        datasets_not_present = []
        datasets_problematic = []
        weighted_coords = []

        try:
            # Plotting the county map
            fig, ax = plt.subplots(figsize=(10, 10))
            county_gdf.plot(ax=ax, color='lightgrey', edgecolor='black')

            total_points = 0

            # Process each dataset
            for path, info in data.items():
                try:
                    gdf = info['gdf']
                    # Ensure the geometries are valid
                    gdf = gdf[gdf.geometry.notnull() & gdf.geometry.is_valid]

                    # Convert 'Weight' to numeric, handling errors
                    if 'Weight' in gdf.columns:
                        gdf['Weight'] = pd.to_numeric(gdf['Weight'], errors='coerce')

                    # Filter points within the county's boundary
                    gdf_in_county = gpd.sjoin(gdf, county_gdf, how="inner", op='within')
                    total_points += len(gdf_in_county)

                    if not gdf_in_county.empty:
                        datasets_present.append(os.path.basename(path))

                        # Append to state-wide facility data
                        all_state_facilities = pd.concat([all_state_facilities, gdf_in_county])

                        # Plot general facilities
                        for geom in gdf_in_county.geometry:
                            if geom.geom_type == 'Point':
                                ax.scatter(geom.x, geom.y, color=info['color'], s=5)
                            elif geom.geom_type == 'LineString':
                                ax.plot(*geom.xy, color=info['color'])
                            elif geom.geom_type == 'MultiLineString':
                                for line in geom.geoms:
                                    ax.plot(*line.xy, color=info['color'])
                            elif geom.geom_type == 'Polygon':
                                ax.add_patch(plt.Polygon(list(geom.exterior.coords), color=info['color'], alpha=0.5))
                            elif geom.geom_type == 'MultiPolygon':
                                for poly in geom.geoms:
                                    ax.add_patch(plt.Polygon(list(poly.exterior.coords), color=info['color'], alpha=0.5))

                        # Aggregate weighted coordinates for later processing in Cell 7
                        for geom, weight in zip(gdf_in_county.geometry, gdf_in_county['Weight']):
                            if geom.geom_type == 'Point':
                                weighted_coords.append((geom.x, geom.y, weight))
                            elif geom.geom_type in ['LineString', 'Polygon', 'MultiLineString', 'MultiPolygon']:
                                centroid = geom.centroid
                                weighted_coords.append((centroid.x, centroid.y, weight))

                    else:
                        datasets_not_present.append(os.path.basename(path))
                except Exception as e:
                    datasets_problematic.append((os.path.basename(path), str(e)))
                    print(f"Error processing dataset {os.path.basename(path)} for {county_name}, {state_name}: {e}")

            # Set the limits for the plot to focus on the county
            minx, miny, maxx, maxy = county_gdf.total_bounds
            ax.set_xlim(minx, maxx)
            ax.set_ylim(miny, maxy)

            # Title and labels with capitalized names
            plt.title(f'Critical Infrastructure in {county_name}, {state_name}', fontsize=14)
            plt.xlabel('Longitude')
            plt.ylabel('Latitude')

            # Save the county map with capitalized names
            county_output_path = f"{county_dir}/{county_name.replace(' ', '_')}_county_map.png"
            plt.savefig(county_output_path)
            plt.close(fig)
            print(f"Saved county map for {county_name}, {state_name}. Total points: {total_points}")

            # **Top 20 Locations by Weight in County**
            if 'Weight' in all_state_facilities.columns and not all_state_facilities['Weight'].isna().all():
                top_20_weighted_county = all_state_facilities[all_state_facilities['Weight'].notna()].nlargest(20, 'Weight')
                fig_top_20_county, ax_top_20_county = plt.subplots(figsize=(10, 10))
                county_gdf.plot(ax=ax_top_20_county, color='lightgrey', edgecolor='black')

                for geom, weight in zip(top_20_weighted_county.geometry, top_20_weighted_county['Weight']):
                    if geom.geom_type == 'Point':
                        ax_top_20_county.scatter(geom.x, geom.y, color='red', s=20 + weight * 5, alpha=0.6)

                # Save the top 20 locations map for the county
                top_20_county_output_path = f"{county_dir}/top_20_facilities_{county_name.replace(' ', '_')}.png"
                plt.savefig(top_20_county_output_path)
                plt.close(fig_top_20_county)
                print(f"Saved top 20 locations map for {county_name}, {state_name}.\n")

            # **Top 5 Aggregate Points for County**
            if len(weighted_coords) > 0:
                weighted_coords_df = pd.DataFrame(weighted_coords, columns=['x', 'y', 'weight'])
                weighted_coords_df['rounded_x'] = weighted_coords_df['x'].round(4)
                weighted_coords_df['rounded_y'] = weighted_coords_df['y'].round(4)
                aggregated_points_county = weighted_coords_df.groupby(['rounded_x', 'rounded_y']).agg({
                    'x': 'first', 'y': 'first', 'weight': 'sum'
                }).nlargest(5, 'weight')

                fig_agg_county, ax_agg_county = plt.subplots(figsize=(10, 10))
                county_gdf.plot(ax_agg_county, color='lightgrey', edgecolor='black')
                ax_agg_county.scatter(aggregated_points_county['x'], aggregated_points_county['y'],
                                      s=20 + aggregated_points_county['weight'] * 5, color='blue', alpha=0.6)

                # Save the top 5 aggregate points map for the county
                agg_county_output_path = f"{county_dir}/top_5_aggregate_{county_name.replace(' ', '_')}.png"
                plt.savefig(agg_county_output_path)
                plt.close(fig_agg_county)
                print(f"Saved top 5 aggregate points map for {county_name}, {state_name}.\n")

            # Save dataset summary table for the county
            summary_data = {
                "Datasets Present": datasets_present,
                "Datasets Not Present": datasets_not_present,
                "Problematic Datasets": [f"{item[0]}: {item[1]}" for item in datasets_problematic]
            }
            df_summary = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in summary_data.items()]))
            summary_output_path = f"{county_dir}/{county_name.replace(' ', '_')}_summary.txt"
            with open(summary_output_path, 'w') as f:
                f.write(tabulate(df_summary, headers="keys", tablefmt="grid"))
            print(f"Saved summary table for {county_name}, {state_name}.\n")

            # Clear memory for this county
            del county_gdf, gdf_in_county, gdf
            plt.close('all')
            gc.collect()

            # Update processed counties count and calculate elapsed time
            processed_counties += 1
            processed_counties_in_state += 1
            county_end_time = time.time()
            county_elapsed_time = timedelta(seconds=(county_end_time - county_start_time))
            state_elapsed_time = time.time() - state_start_time
            average_time_per_county = state_elapsed_time / processed_counties_in_state
            remaining_counties_in_state = len(counties_list) - processed_counties_in_state
            estimated_remaining_time_state = remaining_counties_in_state * average_time_per_county
            remaining_counties_total = total_counties - processed_counties
            estimated_remaining_time_total = remaining_counties_total * average_time_per_county
            print(f"Processing time for {county_name}, {state_name}: {county_elapsed_time}")
            print(f"Estimated remaining time for the state: {timedelta(seconds=estimated_remaining_time_state)}")
            print(f"Estimated remaining time for all counties: {timedelta(seconds=estimated_remaining_time_total)}\n")

        except Exception as e:
            print(f"Error processing county {county_name} in {state_name}: {e}")

    # Save state data for use in Cell 7
    all_state_facilities.to_pickle(f"{state_dir}/state_data.pkl")
    np.save(f"{state_dir}/weighted_coords_state.npy", weighted_coords_state)
    print(f"Saved state data for {state_name}.\n")

print("All county maps created.\n")


In [ ]:
# Cell 7: State-Wide Processing
print("\nStarting to generate state-wide maps...\n")

# Load the US counties dataset for state boundaries
try:
    counties = gpd.read_file("/datasets/raw/geographic/gadm41_USA_2.json")
    if counties.crs != "EPSG:4326":
        counties = counties.to_crs("EPSG:4326")
    print("US counties boundaries loaded successfully.")
except Exception as e:
    print(f"Error loading US counties boundaries: {e}")

# Loop through each state
for state_name, counties_list in states_counties.items():
    # Capitalize state name
    state_name = state_name.title()

    print(f"\nProcessing state-wide maps for: {state_name}...\n")

    # Load saved state data
    state_dir = f"/geoJSON/maps/state_maps/{state_name.replace(' ', '_')}"
    state_data_path = f"{state_dir}/state_data.pkl"
    weighted_coords_state_path = f"{state_dir}/weighted_coords_state.npy"

    try:
        all_state_facilities = pd.read_pickle(state_data_path)
        weighted_coords_state = np.load(weighted_coords_state_path, allow_pickle=True)

        # Create a GeoDataFrame for the current state
        state_gdf = counties[counties['NAME_1'].str.title() == state_name]

        # Create the state-wide maps directory
        whole_state_dir = f"{state_dir}/_whole_state_maps_"
        os.makedirs(whole_state_dir, exist_ok=True)

        # **Whole State Map for All Facilities**
        fig_state_all, ax_state_all = plt.subplots(figsize=(10, 10))
        state_gdf.plot(ax=ax_state_all, color='lightgrey', edgecolor='black')

        for path, info in data.items():
            try:
                gdf = info['gdf']
                gdf_in_state = gpd.sjoin(gdf, state_gdf, how="inner", op='within')
                if not gdf_in_state.empty:
                    for geom in gdf_in_state.geometry:
                        if geom.geom_type == 'Point':
                            ax_state_all.scatter(geom.x, geom.y, color=info['color'], s=5)
                        elif geom.geom_type == 'LineString':
                            ax_state_all.plot(*geom.xy, color=info['color'])
                        elif geom.geom_type == 'MultiLineString':
                            for line in geom.geoms:
                                ax_state_all.plot(*line.xy, color=info['color'])
                        elif geom.geom_type == 'Polygon':
                            ax_state_all.add_patch(plt.Polygon(list(geom.exterior.coords), color=info['color'], alpha=0.5))
                        elif geom.geom_type == 'MultiPolygon':
                            for poly in geom.geoms:
                                ax_state_all.add_patch(plt.Polygon(list(poly.exterior.coords), color=info['color'], alpha=0.5))

            except Exception as e:
                print(f"Error processing dataset {path} for state-level map of {state_name}: {e}")

        # Save the whole state facilities map
        state_all_output_path = f"{whole_state_dir}/all_facilities_{state_name.replace(' ', '_')}.png"
        plt.savefig(state_all_output_path)
        plt.close(fig_state_all)
        print(f"Saved whole state map for all facilities in {state_name}.\n")

        # **Top 20 Locations by Weight in State**
        if 'Weight' in all_state_facilities.columns and not all_state_facilities['Weight'].isna().all():
            top_20_weighted_state = all_state_facilities[all_state_facilities['Weight'].notna()].nlargest(20, 'Weight')
            fig_top_20_state, ax_top_20_state = plt.subplots(figsize=(10, 10))
            state_gdf.plot(ax=ax_top_20_state, color='lightgrey', edgecolor='black')

            for geom, weight in zip(top_20_weighted_state.geometry, top_20_weighted_state['Weight']):
                if geom.geom_type == 'Point':
                    ax_top_20_state.scatter(geom.x, geom.y, color='red', s=20 + weight*5, alpha=0.6)

            # Save the top 20 locations map for the state
            top_20_state_output_path = f"{whole_state_dir}/top_20_facilities_{state_name.replace(' ', '_')}.png"
            plt.savefig(top_20_state_output_path)
            plt.close(fig_top_20_state)
            print(f"Saved top 20 locations map for {state_name}.\n")

        # **Top 5 Aggregate State-Wide Locations**
        if len(weighted_coords_state) > 0:
            weighted_coords_state_df = pd.DataFrame(weighted_coords_state, columns=['x', 'y', 'weight'])

            # Group by rounded coordinates and find top 5 aggregate locations
            weighted_coords_state_df['rounded_x'] = weighted_coords_state_df['x'].round(4)
            weighted_coords_state_df['rounded_y'] = weighted_coords_state_df['y'].round(4)
            aggregated_points_state = weighted_coords_state_df.groupby(['rounded_x', 'rounded_y']).agg({
                'x': 'first', 'y': 'first', 'weight': 'sum'
            }).nlargest(5, 'weight')

            fig_state_agg_5, ax_state_agg_5 = plt.subplots(figsize=(10, 10))
            state_gdf.plot(ax=ax_state_agg_5, color='lightgrey', edgecolor='black')

            ax_state_agg_5.scatter(aggregated_points_state['x'], aggregated_points_state['y'], color='blue', s=100, alpha=0.8)

            # Save the top 5 aggregate state-wide locations map
            state_top_5_agg_output_path = f"{whole_state_dir}/top_5_aggregate_{state_name.replace(' ', '_')}.png"
            plt.savefig(state_top_5_agg_output_path)
            plt.close(fig_state_agg_5)
            print(f"Saved top 5 aggregate points map for {state_name}.\n")

    except Exception as e:
        print(f"Error processing state {state_name} for state-wide maps: {e}")

print("All state-wide maps created.\n")
